## Setup

In [1]:
import math
import numpy as np
from sklearn.utils import class_weight
import tensorflow as tf

In [2]:
def shuffle(a, b, seed):
    'Shuffle two array in the same random order'
    rand_state = np.random.RandomState(seed)
    rand_state.shuffle(a)
    rand_state.seed(seed)
    rand_state.shuffle(b)

In [3]:
def generator(sample_ID_list, labels, batch_size):
    'Yield training data if model.fit_generator request it'
    # Init output arrays
    batch_features = np.zeros((batch_size, 15, 15, 13))
    batch_labels = np.zeros((batch_size,1))
    
    # Randomly shuffle samples (training order changes in each epoch)
    rand_state = np.random.RandomState(13991)
    rand_state.shuffle(sample_ID_list)
    
    # Prepare training data packets
    index = 0
    while True:
        # Generate IDs of the batch
        start_index = index*batch_size
        end_index = (index+1)*batch_size
        list_IDs_temp = sample_ID_list[start_index:end_index]
        
        # Load data from the selected sample files
        for i, ID in enumerate(list_IDs_temp):
            batch_features[i, ] = np.load('data/ID' + str(ID) + '.npy')
            batch_labels[i] = labels[ID]
        
        yield batch_features, batch_labels
        index = index + 1

In [4]:
dummy_data = np.zeros((46, 501, 497, 13))
sample_segment_dim = (15,15)

# Calculate number of samples
sample_number_per_column = dummy_data.shape[1] - sample_segment_dim[0] + 1
sample_number_per_row = dummy_data.shape[2] - sample_segment_dim[1] + 1
#sample_number = dummy_data.shape[0] * sample_number_per_column * sample_number_per_row
sample_number = 100
print(sample_number)

100


In [5]:
ID_list = np.arange(sample_number)
labels = np.zeros(sample_number)

In [6]:
# Creating sample patches from the big images
# Change this code to snip samples from the images!
dummy_sample = np.zeros((15,15,13))
for x in range(0, sample_number):
    np.save('data/ID' + str(ID_list[x]) + '.npy', dummy_sample)

In [7]:
# Creating the Training and Validation sets
shuffle(ID_list, labels, 12045)

validation_split = 0.2
split_index = int((1-validation_split) * sample_number)

train_ID_list = ID_list[:split_index]
validation_ID_list = ID_list[split_index:]

In [8]:
# Create NN model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(4, (3,3), strides=(1, 1), input_shape=(15,15,13)))
print(model.output_shape)
model.add(tf.keras.layers.Flatten())
print(model.output_shape)
model.add(tf.keras.layers.Dense(16))
print(model.output_shape)
model.add(tf.keras.layers.Dense(3, activation="softmax"))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

(None, 13, 13, 4)
(None, 676)
(None, 16)


In [9]:
# Train model on dataset
train_batch_size = 21
validation_batch_size = 11

model.fit_generator(generator(train_ID_list, labels, train_batch_size),
                    steps_per_epoch=math.ceil(split_index/train_batch_size),
                    epochs=3,
                    validation_data=generator(validation_ID_list, labels, validation_batch_size),
                    validation_steps=math.ceil(validation_split*sample_number/validation_batch_size))

Epoch 1/3
4/4 [==============================] - 1s 158ms/step - loss: 1.0842 - val_loss: 1.0482
Epoch 2/3
4/4 [==============================] - 0s 19ms/step - loss: 1.0093 - val_loss: 0.9295
Epoch 3/3
4/4 [==============================] - 0s 16ms/step - loss: 0.8645 - val_loss: 0.7442
